# 脚本文档：

In [1]:
import rdkit
from rdkit import Chem
import pandas as pd
from rdkit.Chem import MACCSkeys

In [7]:
#计算出CEP的MACCS指纹并且存到DataFrame里面
CEP="CN1CCC2=C(C=C(OC3=C([C@@](CC4=CC=C5C=C4)([H])N(C)CC6)C6=CC7=C3OCO7)C(OC)=C2)[C@@]1([H])CC8=CC=C(OC)C(O5)=C8"
mol = Chem.MolFromSmiles(CEP)
fps = MACCSkeys.GenMACCSKeys(mol)
head=['MACCS_'+str(i) for i in range(1,168)]
empty_dict=dict(list(zip(head,list(fps))))
MACCS_CEP=pd.DataFrame(empty_dict,index=["CEP"])

In [8]:
#将需要计算相似度的分子从文件中提取出来
path=r"D:\RNN_folder\high_acticity.csv"
dataset=pd.read_csv(path,encoding="GBK")
smiles_list=list(dataset["SMILES"])

In [9]:
#定义计算函数，输入为MACCS_CEP（DataFrame格式），DataFrame文本格式，返回值为分子与CEP的tanimoto系数
def compute(MACCS_CEP,smiles):
    mol = Chem.MolFromSmiles(smiles)
    fps = MACCSkeys.GenMACCSKeys(mol)
    head=['MACCS_'+str(i) for i in range(1,168)]
    empty_dict=dict(list(zip(head,list(fps))))
    MACCS_smiles=pd.DataFrame(empty_dict,index=["smiles"])
    final=MACCS_smiles.append(MACCS_CEP)
    c=0
    for i in head:
        if final[i]["CEP"]==final[i]["smiles"]:
            c=c+1
    return c/(167+167-c)

In [10]:
#调用函数，计算tanimoto系数，并且存储到原来的数据中
tanimoto_list=[]
for smiles in smiles_list:
    tanimoto=compute(MACCS_CEP,smiles)
    tanimoto_list.append(tanimoto)
dataset["tanimoto"]=tanimoto_list

In [83]:
#将数据写入文件
out_path=path[:-4]+'_tanimoto.csv'
dataset.to_csv(out_path,index=False)

In [11]:
dataset

,SMILES,tanimoto
0,FC(F)(F)C=1/C(=N\c2cc(/N=C(/O)\C=C)ccc2)/NC(Nc...,0.560748
1,FC(F)(F)C=1/C(=N\c2cc(/N=C(/O)\C=C)ccc2)/NC(Nc...,0.553488
2,O=C(Nc1c(OCC)cc2ncnc(Nc3cc(C#C)ccc3)c2c1)/C=C/...,0.670000
3,O=C(Nc1c(OC)cc2ncnc(Nc3cc(C#C)ccc3)c2c1)/C=C/C...,0.678392
4,Clc1c(OCc2ncccc2)ccc(Nc2c(C#N)cnc3c2cc(NC(=O)/...,0.605769
...,...,...
252,O=C(Nc1c(OC)cc2ncnc(Nc3cc(C#C)ccc3)c2c1)/C=C/C...,0.590476
253,O=C(Nc1c(OCC)cc2ncnc(Nc3cc(C#C)ccc3)c2c1)/C=C/...,0.590476
254,O=C(Nc1c(O[C@@H]2COCC2)cc2ncnc(Nc3cc(C#C)ccc3)...,0.670000
255,O=C(OCC)CCCN1C(=O)Oc2c1cc1c(Nc3c(O)ccc(C(=O)OC...,0.695431
